In [1]:
import numpy as np
import pandas as pd

from glob import glob
from os import path

In [2]:
ABCD_SSD_dists = pd.read_csv('abcd_data/SSD_dist_by_subj.csv')
ABCD_SSD_dists

,NARGUID,SSDDur,proportion
0,003RTV85,300.0,0.250000
1,003RTV85,250.0,0.183333
2,003RTV85,400.0,0.183333
3,003RTV85,350.0,0.166667
4,003RTV85,450.0,0.100000
...,...,...,...
62441,xxmy9wd8,100.0,0.183333
62442,xxmy9wd8,150.0,0.133333
62443,xxmy9wd8,200.0,0.116667
62444,xxmy9wd8,250.0,0.100000


In [3]:
indiv_SSRTs = pd.DataFrame()
for file in [i for i in glob('ssrt_metrics/indiv_*.csv') if 'linear' not in i and 'guess' not in i]:
    data =  pd.read_csv(file, index_col=0)
    gen_str = file.split('indiv_')[-1].replace('.csv', '')
    indiv_ssrt_df = pd.DataFrame(np.nan, index=ABCD_SSD_dists.NARGUID.unique(), columns=['vanilla', 'w_guesses', 'w_graded'])
    for sub in ABCD_SSD_dists.NARGUID.unique():
        curr_sub = ABCD_SSD_dists.query("NARGUID=='%s'" % sub)
#         if curr_sub.SSDDur.max() <= 550: #KLUDGE WHILE ALL SSDS ARE BEING SIMULATED
        indiv_ssrt_df.loc[sub, :] = [0, 0, 0]
        for SSD in curr_sub.SSDDur:
            ssd_SSRTs = data.loc[data.SSD==SSD, ['SSRT', 'SSRT_w_guesses', 'SSRT_w_graded']].values[0]
            weight = curr_sub.loc[curr_sub.SSDDur==SSD, 'proportion'].values
            indiv_ssrt_df.loc[sub, :] +=  ssd_SSRTs * weight
    indiv_SSRTs = pd.concat([indiv_SSRTs, indiv_ssrt_df.rank().add_prefix('gen-%s_SSRT-' % gen_str)], 1)


In [4]:
indiv_SSRTs.corr()

,gen-vanilla_SSRT-vanilla,gen-vanilla_SSRT-w_guesses,gen-vanilla_SSRT-w_graded,gen-graded_mu_go_log_SSRT-vanilla,gen-graded_mu_go_log_SSRT-w_guesses,gen-graded_mu_go_log_SSRT-w_graded
gen-vanilla_SSRT-vanilla,1.000000,-0.985365,0.985693,-0.880356,-0.994322,0.988167
gen-vanilla_SSRT-w_guesses,-0.985365,1.000000,-0.933528,0.754218,0.985315,-0.952797
gen-vanilla_SSRT-w_graded,0.985693,-0.933528,1.000000,-0.899832,-0.977456,0.995005
gen-graded_mu_go_log_SSRT-vanilla,-0.880356,0.754218,-0.899832,1.000000,0.876090,-0.886810
gen-graded_mu_go_log_SSRT-w_guesses,-0.994322,0.985315,-0.977456,0.876090,1.000000,-0.983343
gen-graded_mu_go_log_SSRT-w_graded,0.988167,-0.952797,0.995005,-0.886810,-0.983343,1.000000


In [5]:
data.loc[data.SSD==SSD, ['SSRT', 'SSRT_w_guesses', 'SSRT_w_graded']].values

array([[215., 200., 285.]])

In [6]:
SSD

300.0

In [7]:
data

,SSRT,mean_SSD,p_respond,max_RT,mean_go_RT,mean_stopfail_RT,omission_count,omission_rate,go_acc,stopfail_acc,SSD,SSRT_w_guesses,SSRT_w_graded
0,NaN,0.0,0.00000,1441.0,540.16102,NaN,0,0.0,NaN,NaN,0.0,14.545501,NaN
1,222.0,50.0,0.00092,1441.0,540.16102,457.586957,0,0.0,NaN,NaN,50.0,24.618337,456.0
2,222.0,100.0,0.01065,1441.0,540.16102,459.094836,0,0.0,NaN,NaN,100.0,46.776732,347.0
3,216.0,150.0,0.04778,1441.0,540.16102,464.221013,0,0.0,NaN,NaN,150.0,70.095106,325.0
4,214.0,200.0,0.13890,1441.0,540.16102,476.391937,0,0.0,NaN,NaN,200.0,137.554622,304.0
5,213.0,250.0,0.28382,1441.0,540.16102,492.013107,0,0.0,NaN,NaN,250.0,181.000000,302.0
6,215.0,300.0,0.46360,1441.0,540.16102,507.320298,0,0.0,NaN,NaN,300.0,200.000000,285.0
7,222.0,350.0,0.64604,1441.0,540.16102,520.838091,0,0.0,NaN,NaN,350.0,216.000000,291.0
8,230.0,400.0,0.79028,1441.0,540.16102,529.837412,0,0.0,NaN,NaN,400.0,227.000000,283.0
9,241.0,450.0,0.88778,1441.0,540.16102,534.665165,0,0.0,NaN,NaN,450.0,241.000000,267.0


In [8]:
file

'ssrt_metrics/indiv_graded_mu_go_log.csv'

In [9]:
indiv_SSRTs

,gen-vanilla_SSRT-vanilla,gen-vanilla_SSRT-w_guesses,gen-vanilla_SSRT-w_graded,gen-graded_mu_go_log_SSRT-vanilla,gen-graded_mu_go_log_SSRT-w_guesses,gen-graded_mu_go_log_SSRT-w_graded
003RTV85,1392.0,5737.5,1309.5,1890.0,5367.0,1207.0
00CY2MDM,6390.5,350.5,NaN,NaN,339.5,NaN
00HEV6HB,5522.5,1996.0,NaN,NaN,2128.0,NaN
00LJVZK2,4738.0,2353.0,NaN,NaN,3031.0,NaN
00NPMHND,142.0,6606.0,101.0,3397.0,6608.0,137.0
...,...,...,...,...,...,...
hkfm1ruj,1972.5,4066.0,NaN,NaN,5088.0,NaN
hx1ru4hv,5528.0,1406.0,NaN,NaN,1328.0,NaN
jf8w3pw6,5117.0,1530.0,NaN,NaN,1541.0,NaN
x8k59,4510.0,2218.0,NaN,NaN,2248.0,NaN
